<a href="https://colab.research.google.com/github/baroodb/code/blob/main/flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import  Path

In [2]:
url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

In [3]:
data_path = tf.keras.utils.get_file(origin=url, fname='flower_photos', untar=True)
print(data_path)

228827136/228813984 [==============================] - 3s 0us/step
/root/.keras/datasets/flower_photos


In [4]:
path = Path(data_path)
path

PosixPath('/root/.keras/datasets/flower_photos')

In [5]:
entries = path.glob(pattern='*')
for entry in entries:
  if os.path.isfile(entry):
    Path(entry).unlink()

In [6]:
class_names = os.listdir(path)
class_names

['daisy', 'roses', 'dandelion', 'tulips', 'sunflowers']

In [7]:
list_ds = tf.data.Dataset.list_files(str(path/'*/*'), shuffle=False)
list_ds

<TensorSliceDataset shapes: (), types: tf.string>

In [8]:
def _get_labels(filename):
  parts = tf.strings.split(filename, os.path.sep)
  onehot = parts[-2] == class_names
  return tf.argmax(onehot)

def _process_image(filename):
  image = tf.io.read_file(filename)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.resize(image, [224, 224])
  return image

def process_path(filename):
  label = _get_labels(filename)
  image = _process_image(filename)

  return image, label

def configure_for_perf(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=5000)
  ds = ds.batch(32)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

  return ds

def get_datasets(path, val_size=400):
  filenames = tf.data.Dataset.list_files(file_pattern=str(path/'*/*'), shuffle=False)
  filenames = filenames.shuffle(buffer_size=5000, reshuffle_each_iteration=False)
  train_files = filenames.skip(val_size)
  val_files = filenames.take(val_size)
  return train_files, val_files

In [9]:
# build my  data pipeline 
train_files, val_files = get_datasets(path)

train_ds = train_files.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_files.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = configure_for_perf(train_ds)
val_ds = configure_for_perf(val_ds)

In [10]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

def build_model(input_shape):
  inputs = Input(input_shape)
  x = Conv2D(128, (3, 3), activation='relu')(inputs)
  x = MaxPool2D()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPool2D()(x)
  x = Dropout(0.2)(x)

  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPool2D()(x)
  x = Dropout(0.2)(x)
  
  x = Flatten()(x)
  x = Dense(64, activation='relu')(x)
  
  outputs = Dense(5)(x)

  model = Model(inputs=inputs, outputs=outputs)

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'], 
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

  return model

In [11]:
model = build_model((224, 224, 3))

H = model.fit(train_ds, 
              validation_data=val_ds,
              epochs=10)

Epoch 1/10
103/103 [==============================] - 70s 309ms/step - loss: 1.4348 - accuracy: 0.3645 - val_loss: 1.2168 - val_accuracy: 0.4700
Epoch 2/10
103/103 [==============================] - 28s 277ms/step - loss: 1.1224 - accuracy: 0.5382 - val_loss: 1.0700 - val_accuracy: 0.5675
Epoch 3/10
103/103 [==============================] - 28s 276ms/step - loss: 1.0440 - accuracy: 0.5856 - val_loss: 1.0330 - val_accuracy: 0.5650
Epoch 4/10
103/103 [==============================] - 29s 277ms/step - loss: 0.9159 - accuracy: 0.6511 - val_loss: 0.9188 - val_accuracy: 0.6525
Epoch 5/10
103/103 [==============================] - 29s 277ms/step - loss: 0.7047 - accuracy: 0.7275 - val_loss: 0.9279 - val_accuracy: 0.6450
Epoch 6/10
103/103 [==============================] - 29s 277ms/step - loss: 0.4834 - accuracy: 0.8300 - val_loss: 1.0826 - val_accuracy: 0.6400
Epoch 7/10
103/103 [==============================] - 29s 277ms/step - loss: 0.3136 - accuracy: 0.8936 - val_loss: 1.2880 - val_ac